In [ ]:
import six, math

In [2]:
DATA = """
1009310
19,x,x,x,x,x,x,x,x,x,x,x,x,37,x,x,x,x,x,599,x,29,x,x,x,x,x,x,x,x,x,x,x,x,x,x,17,x,x,x,x,x,23,x,x,x,x,x,x,x,761,x,x,x,x,x,x,x,x,x,41,x,x,13
""".strip()

In [3]:
# The time step after which the first bus to arrive is to be determined;
startTimeStep = int(DATA.splitlines()[0].strip())

In [4]:
# The schedule of buses based on their index and interval;
schedule = DATA.splitlines()[1].strip().split(',')
schedule = {t: int(bus) for t, bus in enumerate(schedule) if bus.isdigit()}

In [5]:
def lcm(values):
    """
    Returns the least common multiple of the given values; as found at
    https://stackoverflow.com/questions/37237954/calculate-the-lcm-of-a-list-of-given-numbers-in-python.
    
    :param values: list(int,)
    :return: int
    """
    z = values[0]
    for i in values[1:]:
        z = z * i // math.gcd(z, i)
    return z

In [6]:
def findFirstBusToArriveAfter(schedule, timestep):
    """
    Given a schedule of buses and a timestep, it determines
    the first bus to arrive after the timestep; then returns
    the product of the waiting time after the start timestep
    and the bus identifier.
    
    :param schedule: dict(butIndex:int, busFrequency:int)
    :param timestep: int
    :return: int
    """
    arrivalTimeSteps = {}
    for bus in six.itervalues(schedule):
        currentTimeStep = 0
        while currentTimeStep < startTimeStep:
            currentTimeStep += bus
        arrivalTimeSteps[bus] = currentTimeStep
    earliestArrivalTime = min(six.itervalues(arrivalTimeSteps))
    earliestBus = next(iter(bus for bus, arrivalTimeStep in six.iteritems(arrivalTimeSteps) if arrivalTimeStep == earliestArrivalTime))
    return (earliestArrivalTime - startTimeStep) * earliestBus

In [7]:
findFirstBusToArriveAfter(schedule, startTimeStep)

2995

In [8]:
def findTimeStepMatchingPattern(schedule):
    
    """
    Given a schedule of buses, it determines when the first occurrence
    is of them arriving sequentially one timestep apart from the preceding
    one (i.e. first bus arrives at t=n, the second at t=n+1 and so on).
    
    :param schedule: dict(busIndex:int, busFrequency:int)
    :return: int
    """
    
    # To solve this, timesteps will be iterated over; however, for
    # performance purposes, the increment cannot be the unit, but rather
    # an increasing one.
    
    # The increment starts by being the arrival frequency of the first bus,
    # which implies that at every iteration it is known the first bus will be
    # arriving. However, at each iteration it is determined whether any other
    # bus would be arriving according to the required sequential pattern
    # and if that is the case, this is recorded and the increment increased.
    
    # For instance, if at a certain timestep the first bus is known to arrive,
    # but at the next step the second bus will be arriving, then the increment
    # can be updated to be the least common multiple (or product if bus frequencies
    # are coprimes) of the current increment and the interval for the newly found
    # matching bus. At the next step, then both the first and second are guranteed
    # to arrive.
    
    # The intuition behind this is essentially about first learning for sure
    # when each of the previously seen buses will arrive, based on that
    # tuning expectations based on how long to wait and hoping that at one
    # of this known future timesteps new buses will be found.
    
    intervals = [schedule[0]] # The timestep intervals of accounted buses;
    processed = {0}           # The buses accounted for in the timestep intervals;
    timeStep = 0              # The current timestep (for the first bus) being inspected;
    
    interval = lcm(intervals) # The initial increment (i.e. the first bus's frequency);
    
    while len(processed) < len(schedule):
        # Until a time step has not been found for all of buses;
        timeStep += interval # Increase the timestep by the common interval;
        for i, bus in six.iteritems(schedule):
            # For every bus with a known arrival frequency;
            if i in processed:
                # If the bus was previously encountered, disregard it;
                continue
            elif (timeStep + i) % bus != 0:
                # Otherwise, if the bus would not arrive at the required timestep, disregard it;
                continue
            else:
                # Otherwise, the bus would arrive at the required timestep, that is,
                # the time step of the first bus plus the difference in their position
                # in the schedule matches the expected value; add it to the set of
                # accounted buses, include its interval in the list of bus arrival
                # intervals and determine the overall arrival frequency (for the buses
                # encountered so far) to be the least-common-multiple of the arrival times
                # of the buses encountered so far;
                processed.add(i)
                intervals.append(bus)
                interval = lcm(intervals)
                
    return timeStep

In [9]:
findTimeStepMatchingPattern(schedule)

1012171816131114